In [5]:
# First we need to get our large plots of land that we will then pull images from

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from PIL import Image

# After looking over which plots we liked, we came up with these links
links = [
         'https://macrostrat.org/map/#/z=10.8/x=-108.4046/y=41.3516/', 
         'https://macrostrat.org/map/#/z=10.8/x=-103.5518/y=32.1830/', 
         'https://macrostrat.org/map/#/z=10.8/x=-105.6609/y=31.5821/',
         'https://macrostrat.org/map/#/z=10.8/x=-113.4659/y=33.0559/',
         'https://macrostrat.org/map/#/z=10.8/x=-116.4474/y=42.4211/',
         'https://macrostrat.org/map/#/z=10.8/x=-115.2594/y=34.2346/',
         'https://macrostrat.org/map/#/z=10.8/x=-110.1556/y=35.6799/',
         'https://macrostrat.org/map/#/z=10.1/x=-105.6235/y=29.6456/'
        ]

# For our reference, it's just to understand where our plot is coming from
name = [
        'Wyoming', 
        'New_Mexico', 
        'Texas', 
        'Arizona_S', 
        'Idaho', 
        'California',
        'Arizona_N',
        'Chihuahua_N'
       ]

# Process:

# The way that we will be getting our initial images we first did try to
# use api's to try to pick out the images that we wanted. This worked when
# trying to get the pure satellite images, but it proved more challenging to
# get the pure formation images.

# To get these formation images then is to write up a program that will act as
# a web browser, go to the link provided, and take a screenshot. Then instead
# of using the api for the satellite image and this screenshot method for
# the formation, we just went with going the screenshot method for both

# Now the way we got our satellite image and our formation image (referred to as bedrock)
# The way that we could use one link to get both images was the way that the website
# was structured. They made it easy to access by being able to travel from one
# format to the other by doing <Original link>\satellite for the satellite image
# <Original link>\bedrock for formation image and lastly <Original link>\bedrock\satellite
# for both formation and satellite.

extensions = ['satellite', 'bedrock']

idx = 0
driver = webdriver.Chrome(ChromeDriverManager().install())

# So for this loop, we are going through our process for every link in
# the provided links
for link in links:
    
    # Will hold the images that will be saved
    imgs = []
    
    # Here we are getting all the images needed for each link
    # bedrock (formation), satellite, and both
    for ext in extensions:
        linkWithProtocol = str(link)
        driver.set_window_size(2200, 1800)
        driver.get(linkWithProtocol + str(ext))
        time.sleep(7)
        driver.refresh()
        time.sleep(7)
        driver.save_screenshot(str(name[idx]) + "-" + str(ext) + ".png")
        
        imgs.append(Image.open(str(name[idx]) + "-" + str(ext) + ".png"))
    
    # Getting the combined image
    linkWithProtocol = str(link)
    driver.set_window_size(2200, 1800)
    driver.get(linkWithProtocol + "bedrock/satellite")
    time.sleep(7)
    driver.refresh()
    time.sleep(7)
    driver.save_screenshot(str(name[idx]) + "-bedrock-satellite.png")
    imgs.append(Image.open(str(name[idx]) + "-bedrock-satellite.png"))
    
    # Because the screen shot will contain other chunk information
    # such as the search box and the mapbox copyright. So here we are cropping out
    # those parts out along with making it a square image (880x880 pxls)
    
    # These cropped and square images will be stored in here
    altered = []
    
    for img in imgs:
        w, h = img.size
        left = 492
        top = 88
        right = w - 552
        bottom = h
        altered.append(img.crop((left, top, right, bottom)))
    
    # Here we are going to merge the satellite image and the formation image.
    # They will be merged with the satellite image on the left and the 
    # formation image on the right
    gan_img = Image.new('RGB', (altered[0].width + altered[1].width, altered[0].height))
    gan_img.paste(altered[0], (0,0))
    gan_img.paste(altered[1], (altered[0].width, 0))
    
    # Then the two images (the merged and the overlayed) will be saved off
    gan_img.save("Collected_Images/Final/" + str(name[idx]) + "-gan.png")
    altered[2].save("Collected_Images/Final/" + str(name[idx]) + "-overlay.png")
    
    # To clean up space, the cropped images and the individual images will be
    # removed.
    for ext in extensions:
        os.remove(str(name[idx]) + "-" + str(ext) + ".png")
    os.remove(str(name[idx]) + "-bedrock-satellite.png")
    
    
    idx += 1
driver.close()

[WDM] - Cache is valid for [09/06/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - Driver found in cache [C:\Users\lawso\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


In [6]:
# This section will generate "n" amount of randomly selected images from
# the initial land plots generated in the last chunk

import random

# As the input images are 880x880, it made sense to split up the image into
# a 10x10 subplots with each one being 88x88 pxls. So to access a certain
# subplot from the original images is to pick a number from 0 to 9 then multiply
# by 88 for both the x and y dimension.

nums = [0,1,2,3,4,5,6,7,8,9]

# To make sure that we don't have any repeat subplots, we'll store them here
visited = []

path = "Collected_Images/Final/"
out_path = "Collected_Images/Split_Images/"

name = [
        'Wyoming', 
        'New_Mexico', 
        'Texas', 
        'Arizona_S', 
        'Idaho', 
        'California',
        'Arizona_N',
        'Chihuahua_N'
       ]

temp_img = [
        'Wyoming-gan.png', 
        'New_Mexico-gan.png', 
        'Texas-gan.png', 
        'Arizona_S-gan.png', 
        'Idaho-gan.png', 
        'California-gan.png',
        'Arizona_N-gan.png',
        'Chihuahua_N-gan.png'
       ]


num_imgs = 600

for count in range(num_imgs): 
    # First get a random subplot that has not been visited
    # before.
    temp = random.randrange(0, len(temp_img))
    src_num = temp
    temp = random.randrange(0, len(nums))
    x = nums[temp]
    temp = random.randrange(0, len(nums))
    y = nums[temp]
    
    cur = {
        "source":str(name[src_num]),
        "coords":(x,y)
    }
    
    # Here in the case that we have already visited a certain plot
    # then we just keep on looking until we it one that we haven't seen
    while(len(visited) != 0 and cur in visited):
        temp = random.randrange(0, len(temp_img))
        src_num = temp        
        temp = random.randrange(0, len(nums))
        x = nums[temp]
        temp = random.randrange(0, len(nums))
        y = nums[temp]
        
        cur = {
        "source":str(name[src_num]),
        "coords":(x,y)
        }
        
    visited.append(cur)
    
    # Next we want to import the original image and crop out the correct subplot
    # for both the satellite image and the formation image
    img = Image.open(path + temp_img[src_num])
    
    left = 88 * x
    top = 88 * y
    right = left + 80
    bottom = top + 80
    
    lft_img = img.crop((left,
                        top,
                        right,
                        bottom))
    
    rht_img = img.crop((left+880,
                        top,
                        right+880,
                        bottom))
    
    # Lastly combine the two images (subplot of the satellite and the subplot of the
    # formation) into one image.
    joint = Image.new("RGB", (lft_img.width + rht_img.width, lft_img.height))
    joint.paste(lft_img, (0,0))
    joint.paste(rht_img, (lft_img.width, 0))
    
    joint.save(out_path + str(count) + "-" + name[src_num] + "-img.png")